In [1]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [3]:
import pandas as pd

# Load the new dataset from the Excel file
new_dataset_path = 'dataset.xlsx'

# Load the Excel file into a DataFrame
new_dataset_df = pd.read_excel(new_dataset_path)

# Adding word count columns for each personality trait, handling NaN or non-string values
# new_dataset_df['Extraversion_Word_Count'] = new_dataset_df['EXTRAVERSION'].astype(str).apply(lambda x: len(x.split()))
# new_dataset_df['Agreeableness_Word_Count'] = new_dataset_df['AGREEABLENESS'].astype(str).apply(lambda x: len(x.split()))
# new_dataset_df['Conscientiousness_Word_Count'] = new_dataset_df['CONSCIENTIOUSNESS'].astype(str).apply(lambda x: len(x.split()))
# new_dataset_df['Neuroticism_Word_Count'] = new_dataset_df['NEUROTICISM'].astype(str).apply(lambda x: len(x.split()))
# new_dataset_df['Openness_Word_Count'] = new_dataset_df['OPENNESS'].astype(str).apply(lambda x: len(x.split()))

# Drop No Column
df = new_dataset_df.drop("No",  axis=1)

# Display the updated DataFrame to ensure the word counts have been added correctly
df.head()


,EXTRAVERSION,AGREEABLENESS,CONSCIENTIOUSNESS,NEUROTICISM,OPENNESS
0,"Baru saja kenalan sama orang baru di kafe, dan...",Tadi ketemu kucing yang kuruss banget... Kasia...,"Ngeri banget sih pergaulan jaman sekarang, unt...",Kadang tuh mikir kok bisa ada manusia sehampir...,Tolong diubah mindset kalau cuman laki-laki aj...
1,"Gak bisa diem di rumah nih, pengen banget jala...",Semoga kamu lolos seleksi beasiswa yaa,"Sejak kelas 12, aku harus hidup sehat demi kel...","Gue bisa apa ya, kalo gue bisa sesuatu pasti h...",Sesama rakyat itu coba merangkul dengan diskus...
2,"Ngecash energi dengan ketemu orang kantor nih,...",Main sama anak-anak di Sekolah Bahari termasuk...,Hari ini lulus magang mbkm kayak bersyukur ban...,Ada loker yang aku pengenin banget dan aku lag...,Menurut gue pendidikan itu untuk mencerdaskan ...
3,"Hari ini sibuk banget, tapi seneng karena bisa...","Tadi habis bantuin kakek-kakek nyebarang, kena...","pengen cepet-cepet kuliah lagi, kembali ke rut...",Gue takut gagal sih ambil keputusan yag sulit ...,Padahal tiap org punya cara tersendiri buat me...
4,Pesta di rumah tetangga bikin hari jadi gaul b...,Ribet banget deh orang-orang debatin 01 sama 0...,"Besok mau ngapain ya, apa aku harus buat to do...",Sebenarnya gue juga capek tapi mau gimana lagi...,"Gue udah coba, travelling, jalanin hobi baru, ..."


PREPROCESS

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Prepare the data
# Assuming each personality trait column is treated as a separate class (multi-label classification)
X = df[['EXTRAVERSION', 'AGREEABLENESS', 'CONSCIENTIOUSNESS', 'NEUROTICISM', 'OPENNESS']]
y = df[['EXTRAVERSION', 'AGREEABLENESS', 'CONSCIENTIOUSNESS', 'NEUROTICISM', 'OPENNESS']]

# Since y contains text data, we'll encode it into numerical labels
le = LabelEncoder()

# Assuming the classification is based on the first column (you can adjust according to your needs)
y_encoded = le.fit_transform(y.iloc[:, 0])  # You can use any of the columns or combine them as necessary

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


#### Case Folding

In [29]:
# ------ Case Folding --------
# Perform case folding by converting all text columns to lowercase
text_cols = df.select_dtypes(include=['object']).columns
df[text_cols] = df[text_cols].apply(lambda x: x.str.lower())

print(df.head())

                                        EXTRAVERSION  \
0  baru saja kenalan sama orang baru di kafe, dan...   
1  gak bisa diem di rumah nih, pengen banget jala...   
2  ngecash energi dengan ketemu orang kantor nih,...   
3  hari ini sibuk banget, tapi seneng karena bisa...   
4  pesta di rumah tetangga bikin hari jadi gaul b...   

                                       AGREEABLENESS  \
0  tadi ketemu kucing yang kuruss banget... kasia...   
1             semoga kamu lolos seleksi beasiswa yaa   
2  main sama anak-anak di sekolah bahari termasuk...   
3  tadi habis bantuin kakek-kakek nyebarang, kena...   
4  ribet banget deh orang-orang debatin 01 sama 0...   

                                   CONSCIENTIOUSNESS  \
0  ngeri banget sih pergaulan jaman sekarang, unt...   
1  sejak kelas 12, aku harus hidup sehat demi kel...   
2  hari ini lulus magang mbkm kayak bersyukur ban...   
3  pengen cepet-cepet kuliah lagi, kembali ke rut...   
4  besok mau ngapain ya, apa aku harus buat to

#### Tokenizing

In [39]:
import string 
import re #regex library
from nltk.tokenize import word_tokenize

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

# Define preprocessing functions
def remove_tweet_special(text):
    if isinstance(text, str):
        text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
        text = text.encode('ascii', 'replace').decode('ascii')
        text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
        return text.replace("http://", " ").replace("https://", " ")
    return text

def remove_number(text):
    if isinstance(text, str):
        return re.sub(r"\d+", "", text)
    return text

def remove_punctuation(text):
    if isinstance(text, str):
        return text.translate(str.maketrans("", "", string.punctuation))
    return text

def remove_whitespace_LT(text):
    if isinstance(text, str):
        return text.strip()
    return text

def remove_whitespace_multiple(text):
    if isinstance(text, str):
        return re.sub('\s+', ' ', text)
    return text

def remove_singl_char(text):
    if isinstance(text, str):
        return re.sub(r"\b[a-zA-Z]\b", "", text)
    return text

def word_tokenize_wrapper(text):
    if isinstance(text, str):
        return word_tokenize(text)
    return []

# Assuming the text column is the first column in your dataframe
text_column = df.columns[0]
print(text_column)

# Apply preprocessing functions
df[text_column] = df[text_column].apply(remove_tweet_special)
df[text_column] = df[text_column].apply(remove_number)
df[text_column] = df[text_column].apply(remove_punctuation)
df[text_column] = df[text_column].apply(remove_whitespace_LT)
df[text_column] = df[text_column].apply(remove_whitespace_multiple)
df[text_column] = df[text_column].apply(remove_singl_char)
df['tokens'] = df[text_column].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df['tokens'].head())

EXTRAVERSION
Tokenizing Result : 

0    [baru, saja, kenalan, sama, orang, baru, di, k...
1    [gak, bisa, diem, di, rumah, nih, pengen, bang...
2    [ngecash, energi, dengan, ketemu, orang, kanto...
3    [hari, ini, sibuk, banget, tapi, seneng, karen...
4    [pesta, di, rumah, tetangga, bikin, hari, jadi...
Name: tokens, dtype: object


In [40]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

df['tokens_fdist'] = df['tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(df['tokens_fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

0    [(baru, 2), (seru, 2), (saja, 1), (kenalan, 1)...
1    [(gak, 1), (bisa, 1), (diem, 1), (di, 1), (rum...
2    [(ngecash, 1), (energi, 1), (dengan, 1), (kete...
3    [(hari, 1), (ini, 1), (sibuk, 1), (banget, 1),...
4    [(pesta, 1), (di, 1), (rumah, 1), (tetangga, 1...
Name: tokens_fdist, dtype: object


#### Stopwords removal

In [41]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
# txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)

# # convert stopword string to list & append additional stopword
# list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['tokens_fdist'] = df['tokens'].apply(stopwords_removal) 


print(df['tokens_fdist'].head())

0    [kenalan, orang, kafe, seru, banget, intinya, ...
1    [diem, rumah, pengen, banget, jalanjalan, sore...
2    [ngecash, energi, ketemu, orang, kantor, seru,...
3    [sibuk, banget, seneng, ketemu, orang, bertuka...
4    [pesta, rumah, tetangga, gaul, banget, ikutan,...
Name: tokens_fdist, dtype: object


#### Normalization

In [42]:
normalizad_word = pd.read_csv("normalisasi.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['normalized'] = df['tokens'].apply(normalized_term)

print(df['normalized'].head(10))

0    [baru, saja, kenalan, sama, orang, baru, di, k...
1    [tidak, bisa, diem, di, rumah, nih, pengen, ba...
2    [ngecash, energi, dengan, ketemu, orang, kanto...
3    [hari, ini, sibuk, banget, tetapi, seneng, kar...
4    [pesta, di, rumah, tetangga, bikin, hari, jadi...
5    [ulangtahun, adik, hari, ini, seru, banget, pe...
6    [hari, ini, saya, berasa, gaul, abis, dah, han...
7    [hari, ini, saya, kenalan, sama, teman, baru, ...
8    [pagipagi, sudah, optimis, banget, hari, ini, ...
9    [saya, lagi, planning, perjalanan, baru, nih, ...
Name: normalized, dtype: object


#### Stemmer

In [43]:
pip install Sastrawi swifter

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\envs\riset\lib\site-packages\paho_mqtt-1.6.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at c:\programdata\anaconda3\envs\riset\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [44]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['tokens_stemmed'] = df['normalized'].swifter.apply(get_stemmed_term)
print(df['tokens_stemmed'])

1327
------------------------
baru : baru
saja : saja
kenalan : kenal
sama : sama
orang : orang
di : di
kafe : kafe
dan : dan
seru : seru
banget : banget
intinya : inti
ngobrolin : ngobrolin
halhal : halhal
random : random
itu : itu
selalu : selalu
tidak : tidak
bisa : bisa
diem : diem
rumah : rumah
nih : nih
pengen : ken
jalanjalan : jalanjalan
sore : sore
ini : ini
ada : ada
saran : saran
tempat : tempat
keren : keren
guys : guys
ngecash : ngecash
energi : energi
dengan : dengan
ketemu : ketemu
kantor : kantor
bangeeett : bangeeett
hari : hari
sibuk : sibuk
tetapi : tetapi
seneng : neng
karena : karena
banyak : banyak
bertukar : tukar
ide : ide
teamwork : teamwork
makes : makes
the : the
dream : dream
work : work
right : right
pesta : pesta
tetangga : tetangga
bikin : bikin
jadi : jadi
gaul : gaul
semua : semua
ikutan : ikut
shopping : shopping
ceria : ceria
abis : abis
ulangtahun : ulangtahun
adik : adik
temanteman : temanteman
makan : makan
tertawa : tertawa
pelukpelukan : pelukpel

Pandas Apply:   0%|          | 0/330 [00:00<?, ?it/s]

0      [baru, saja, kenal, sama, orang, baru, di, kaf...
1      [tidak, bisa, diem, di, rumah, nih, ken, bange...
2      [ngecash, energi, dengan, ketemu, orang, kanto...
3      [hari, ini, sibuk, banget, tetapi, neng, karen...
4      [pesta, di, rumah, tetangga, bikin, hari, jadi...
                             ...                        
325                                                   []
326                                                   []
327                                                   []
328                                                   []
329                                                   []
Name: tokens_stemmed, Length: 330, dtype: object


In [45]:
df.to_csv("dataset_final_preprocess.csv")

Data Split

In [5]:
import pandas as pd

# Load CSV file
df = pd.read_csv("dataset_final_preprocess.csv")

# Print kolom-kolom yang ada di dalam CSV
print(df.columns)

Index(['Unnamed: 0', 'EXTRAVERSION', 'AGREEABLENESS', 'CONSCIENTIOUSNESS',
       'NEUROTICISM', 'OPENNESS', 'tokens', 'tokens_fdist', 'normalized',
       'tokens_stemmed'],
      dtype='object')


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the preprocessed dataset from CSV file
df = pd.read_csv("dataset_final_preprocess.csv")

# Drop the unnecessary 'Unnamed: 0' column
df = df.drop(['Unnamed: 0'], axis=1)

# Split the data into features (X) and labels (y)
X = df.drop(['tokens_stemmed'], axis=1)
y = df['tokens_stemmed']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the training and testing sets to separate CSV files
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

LSTM

In [25]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout

# Load the preprocessed dataset
df = pd.read_csv("dataset_final_preprocess.csv")

# Split the data into training and testing sets
X = df['tokens']
y = df['tokens_stemmed']  # use one of the personality traits as the target label

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a tokenizer
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

# Convert text data to sequences
x_train = tokenizer.texts_to_sequences(X_train)
x_val = tokenizer.texts_to_sequences(X_val)

# Pad the sequences
maxlen = 200
x_train = pad_sequences(x_train, maxlen=maxlen)
x_val = pad_sequences(x_val, maxlen=maxlen)

# Combine y_train and y_val into a single array
y_all = np.concatenate((y_train, y_val))

# Fit the LabelEncoder on the combined array
le = LabelEncoder()
le.fit(y_all)

# Transform y_train and y_val using the same encoder
y_train = le.transform(y_train)
y_val = le.transform(y_val)

# One-hot encode the target variable
num_classes = len(le.classes_)
y_train = np.eye(num_classes)[y_train]
y_val = np.eye(num_classes)[y_val]

# Create the model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128))
model.add(LSTM(64))  # add an LSTM layer with 64 units
model.add(Dense(32, activation='relu'))  # add a dense layer with 32 units and ReLU activation
model.add(Dropout(0.2))  # add a dropout layer with 20% dropout rate
model.add(Dense(num_classes, activation='softmax'))  # output layer with num_classes classes

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_val, y_val))

# Evaluate the model on the validation set
score = model.evaluate(x_val, y_val, verbose=0)
print("Validation loss:", score[0])
print("Validation accuracy:", score[1])

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.0016 - loss: 5.7594 - val_accuracy: 0.0152 - val_loss: 5.7621
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.0376 - loss: 5.7502 - val_accuracy: 0.0152 - val_loss: 5.7694
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.0413 - loss: 5.7353 - val_accuracy: 0.0152 - val_loss: 5.7926
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.0383 - loss: 5.6956 - val_accuracy: 0.0152 - val_loss: 5.8463
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.0359 - loss: 5.6794 - val_accuracy: 0.0152 - val_loss: 6.0017
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.0144 - loss: 5.6468 - val_accuracy: 0.0152 - val_loss: 6.2061
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.0365 - loss: 5.4790 - val_accuracy: 0.0152 - val_loss: 6.6587
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.0252 - loss: 5.3698 - val_accuracy: 0.0303 - val_loss: 7.0080
